#Estadística Aplicada

Actividad 6 - Parte A

Nombre: Susana Yareli Lino Reséndiz

Matricula: 1986245

Grupo: 41

1. Importa las librerías básicas y especializadas que consideres necesarias

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [36]:
from lifelines import CoxPHFitter
import seaborn as sns
from lifelines import ExponentialFitter
from lifelines import WeibullFitter
from lifelines import LogNormalFitter
from lifelines import LogNormalAFTFitter

2. Importa la base de datos

In [4]:
disersion_laboral = pd.read_csv('https://raw.githubusercontent.com/jimmyzac/Estadistica-Aplicada-FCFM-UANL/main/bases_datos/desercion_laboral.csv')

In [32]:
disersion_laboral.head(10)

,satisfaccion,evaluacion,proyectos,promedio_horas_mes,semestre,accidente,renunciar,ascenso_5years,dep_0,dep_1,dep_2,dep_3,dep_4,dep_5,dep_6,dep_7,dep_8,sal_0,sal_1
0,3.8,5.3,2,157,3,0,1,0,1,0,0,0,0,0,0,0,0,1,0
1,8.0,8.6,5,262,6,0,1,0,1,0,0,0,0,0,0,0,0,0,1
2,1.1,8.8,7,272,4,0,1,0,1,0,0,0,0,0,0,0,0,0,1
3,7.2,8.7,5,223,5,0,1,0,1,0,0,0,0,0,0,0,0,1,0
4,3.7,5.2,2,159,3,0,1,0,1,0,0,0,0,0,0,0,0,1,0
5,4.1,5.0,2,153,3,0,1,0,1,0,0,0,0,0,0,0,0,1,0
6,1.0,7.7,6,247,4,0,1,0,1,0,0,0,0,0,0,0,0,1,0
7,9.2,8.5,5,259,5,0,1,0,1,0,0,0,0,0,0,0,0,1,0
8,8.9,10.0,5,224,5,0,1,0,1,0,0,0,0,0,0,0,0,1,0
9,4.2,5.3,2,142,3,0,1,0,1,0,0,0,0,0,0,0,0,1,0


3. Depurar la base de datos y estadísticas descriptivas



> a) Verificar que no haya missing values, en caso de haberlos decidir imputar o eliminar



In [6]:
disersion_laboral.isnull().sum()

satisfaccion           0
evaluacion             0
proyectos              0
promedio_horas_mes     0
semestre               0
accidente              0
renunciar              0
ascenso_5years         0
departamento          26
salario                0
dtype: int64

In [8]:
disersion_laboral.count()

satisfaccion          14999
evaluacion            14999
proyectos             14999
promedio_horas_mes    14999
semestre              14999
accidente             14999
renunciar             14999
ascenso_5years        14999
departamento          14973
salario               14999
dtype: int64

In [10]:
# Se eliminan los 26 registros ya que hay 14,999, por lo que no se perdera tanta información
disersion_laboral = disersion_laboral.dropna()

In [11]:
disersion_laboral.isnull().sum()

satisfaccion          0
evaluacion            0
proyectos             0
promedio_horas_mes    0
semestre              0
accidente             0
renunciar             0
ascenso_5years        0
departamento          0
salario               0
dtype: int64


> b) Verificar que no haya valores duplicados, en caso de haber valores duplicados eliminarlos



In [12]:
disersion_laboral.duplicated().sum()

2982

In [13]:
#Si hay valores duplicados, por lo que se eliminan
disersion_laboral = disersion_laboral.drop_duplicates(keep='first').reset_index(drop=True)

In [14]:
disersion_laboral.duplicated().sum()

0



> c) Verificar  que  todas  las  variables  sean  numéricas,  si  no  son  numéricas hacerlas   numéricas,   en   el   caso   de   las   categóricas   convertirlas   en dummies y conservar n-1 dummies



In [16]:
disersion_laboral.dtypes

satisfaccion          float64
evaluacion            float64
proyectos               int64
promedio_horas_mes      int64
semestre                int64
accidente               int64
renunciar               int64
ascenso_5years          int64
departamento           object
salario                object
dtype: object

In [21]:
#Variable departamento
disersion_laboral['departamento'].value_counts()

0    3239
1    2244
2    1821
3     976
4     694
5     686
6     673
7     621
8     601
9     436
Name: departamento, dtype: int64

In [18]:
#Departamento
# 0 = ventas
# 1 = mantenimiento
# 2 = soporte
# 3 = informática
# 4 = investigación
# 5 = desarrollo de producto
# 6 = mercadotecnia
# 7 = contabilidad
# 8 = recursos humanos
# 9 = administración
conditions=[disersion_laboral['departamento']=='ventas',
            disersion_laboral['departamento']=='mantenimiento',
            disersion_laboral['departamento']=='soporte',
            disersion_laboral['departamento']=='informática',
            disersion_laboral['departamento']=='Investigación',
            disersion_laboral['departamento']=='desarrollo de producto',
            disersion_laboral['departamento']=='mercadotecnia',
            disersion_laboral['departamento']=='contabilidad',
            disersion_laboral['departamento']=='recursos humanos',
            disersion_laboral['departamento']=='administración']
conditions_values=[0,1,2,3,4,5,6,7,8,9]
disersion_laboral['departamento']=np.select(conditions,conditions_values,default='NA')
disersion_laboral['departamento'].value_counts()

0    3239
1    2244
2    1821
3     976
4     694
5     686
6     673
7     621
8     601
9     436
Name: departamento, dtype: int64

In [20]:
#Crear dummies y no se pone la categoria 9 = administración
dummies_departamento = pd.get_dummies(disersion_laboral['departamento'],prefix='dep')
dummies_departamento = dummies_departamento[['dep_0','dep_1','dep_2','dep_3','dep_4','dep_5','dep_6','dep_7','dep_8']].astype('uint8')
dummies_departamento

,dep_0,dep_1,dep_2,dep_3,dep_4,dep_5,dep_6,dep_7,dep_8
0,1,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
11986,0,0,0,0,0,1,0,0,0
11987,0,0,0,0,0,1,0,0,0
11988,0,0,0,0,0,1,0,0,0
11989,0,0,0,0,0,1,0,0,0


In [22]:
#Variable salario
disersion_laboral['salario'].value_counts()

bajo     5740
medio    5261
alto      990
Name: salario, dtype: int64

In [23]:
#Salarios
# 0 = bajo
# 1 = medio
# 2 = alto
conditions = [disersion_laboral['salario']=='bajo',
              disersion_laboral['salario']=='medio',
              disersion_laboral['salario']=='alto']
conditions_values = [0,1,2]
disersion_laboral['salario'] = np.select(conditions,conditions_values,default='NA')
disersion_laboral['salario'].value_counts()

0    5740
1    5261
2     990
Name: salario, dtype: int64

In [25]:
#Crear dummies y no se pone la categoria 2 = alto
dummies_salarios = pd.get_dummies(disersion_laboral['salario'],prefix='sal')
dummies_salarios = dummies_salarios[['sal_0','sal_1']].astype('uint8')
dummies_salarios

,sal_0,sal_1
0,1,0
1,0,1
2,0,1
3,1,0
4,1,0
...,...,...
11986,1,0
11987,1,0
11988,1,0
11989,1,0


In [26]:
disersion_laboral = pd.concat([disersion_laboral, dummies_departamento, dummies_salarios],axis=1)
disersion_laboral = disersion_laboral.drop(['departamento','salario'],axis=1)
disersion_laboral

,satisfaccion,evaluacion,proyectos,promedio_horas_mes,semestre,accidente,renunciar,ascenso_5years,dep_0,dep_1,dep_2,dep_3,dep_4,dep_5,dep_6,dep_7,dep_8,sal_0,sal_1
0,3.8,5.3,2,157,3,0,1,0,1,0,0,0,0,0,0,0,0,1,0
1,8.0,8.6,5,262,6,0,1,0,1,0,0,0,0,0,0,0,0,0,1
2,1.1,8.8,7,272,4,0,1,0,1,0,0,0,0,0,0,0,0,0,1
3,7.2,8.7,5,223,5,0,1,0,1,0,0,0,0,0,0,0,0,1,0
4,3.7,5.2,2,159,3,0,1,0,1,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11986,3.7,4.7,2,152,3,0,1,0,0,0,0,0,0,1,0,0,0,1,0
11987,8.4,9.9,4,267,5,0,1,0,0,0,0,0,0,1,0,0,0,1,0
11988,4.1,4.6,2,151,3,0,1,0,0,0,0,0,0,1,0,0,0,1,0
11989,7.6,9.2,4,239,5,0,1,0,0,0,0,0,0,1,0,0,0,1,0




> d) Calcular estadísticas descriptivas



In [27]:
disersion_laboral.describe()

,satisfaccion,evaluacion,proyectos,promedio_horas_mes,semestre,accidente,renunciar,ascenso_5years,dep_0,dep_1,dep_2,dep_3,dep_4,dep_5,dep_6,dep_7,dep_8,sal_0,sal_1
count,11991.000000,11991.000000,11991.000000,11991.000000,11991.000000,11991.000000,11991.000000,11991.000000,11991.000000,11991.00000,11991.000000,11991.000000,11991.000000,11991.000000,11991.000000,11991.000000,11991.000000,11991.000000,11991.000000
mean,6.296581,7.166825,3.802852,200.473522,3.364857,0.154282,0.166041,0.016929,0.270119,0.18714,0.151864,0.081394,0.057877,0.057210,0.056125,0.051789,0.050121,0.478692,0.438746
std,2.410700,1.683426,1.163238,48.727813,1.330240,0.361234,0.372133,0.129012,0.444040,0.39004,0.358904,0.273451,0.233520,0.232252,0.230173,0.221610,0.218204,0.499567,0.496254
min,0.900000,3.600000,2.000000,96.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,4.800000,5.700000,3.000000,157.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,6.600000,7.200000,4.000000,200.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,8.200000,8.600000,5.000000,243.000000,4.000000,0.000000,0.000000,0.000000,1.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000
max,10.000000,10.000000,7.000000,310.000000,10.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


4. Estimar el modelode Cox e interpretar cada uno de los coeficientes

In [28]:
CPH = CoxPHFitter().fit(disersion_laboral,'semestre','renunciar')
CPH.print_summary()

<lifelines.CoxPHFitter: fitted with 11991 total observations, 10000 right-censored observations>
             duration col = 'semestre'
                event col = 'renunciar'
      baseline estimation = breslow
   number of observations = 11991
number of events observed = 1991
   partial log-likelihood = -15884.05
         time fit was run = 2023-09-26 21:00:35 UTC

---
                     coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                                  
satisfaccion        -0.24       0.78       0.01            -0.26            -0.22                 0.77                 0.80
evaluacion          -0.01       0.99       0.01            -0.04             0.02                 0.96                 1.02
proyectos           -0.29       0.75       0.02            -0.33            -0.25                 0.72                 0.78
promedio_horas_mes   0.00       1.00       0.00             0.00             0.00                 1.00                 1.00
accidente           -1.25       0.29       0.10            -1.44            -1.05                 0.24                 0.35
ascenso_5years      -1.38       0.25       0.36            -2.08            -0.68                 0.13                 0.51
dep_0                0.13       1.14       0.15            -0.16             0.42                 0.85                 1.52
dep_1                0.21       1.24       0.15            -0.08             0.50                 0.92                 1.66
dep_2                0.23       1.26       0.15            -0.07             0.53                 0.94                 1.69
dep_3                0.07       1.07       0.16            -0.24             0.39                 0.78                 1.47
dep_4               -0.22       0.80       0.18            -0.57             0.13                 0.57                 1.14
dep_5                0.14       1.15       0.17            -0.19             0.47                 0.82                 1.60
dep_6                0.12       1.13       0.17            -0.21             0.45                 0.81                 1.57
dep_7                0.02       1.02       0.17            -0.31             0.35                 0.73                 1.42
dep_8                0.21       1.24       0.17            -0.12             0.55                 0.89                 1.72
sal_0                1.54       4.66       0.15             1.25             1.83                 3.48                 6.23
sal_1                1.15       3.15       0.15             0.85             1.44                 2.35                 4.22

                     cmp to      z      p   -log2(p)
covariate                                           
satisfaccion           0.00 -26.89 <0.005     526.58
evaluacion             0.00  -0.76   0.44       1.17
proyectos              0.00 -14.75 <0.005     161.15
promedio_horas_mes     0.00   3.88 <0.005      13.25
accidente              0.00 -12.41 <0.005     115.04
ascenso_5years         0.00  -3.88 <0.005      13.21
dep_0                  0.00   0.89   0.37       1.42
dep_1                  0.00   1.42   0.16       2.68
dep_2                  0.00   1.53   0.13       2.98
dep_3                  0.00   0.45   0.66       0.61
dep_4                  0.00  -1.24   0.21       2.22
dep_5                  0.00   0.82   0.41       1.29
dep_6                  0.00   0.70   0.48       1.05
dep_7                  0.00   0.12   0.91       0.14
dep_8                  0.00   1.27   0.21       2.29
sal_0                  0.00  10.36 <0.005      81.15
sal_1                  0.00   7.67 <0.005      45.68
---
Concordance = 0.85
Partial AIC = 31802.11
log-likelihood ratio test = 1360.94 on 17 df
-log2(p) of ll-ratio test = 924.90

In [30]:
CPH.params_

covariate
satisfaccion         -0.242337
evaluacion           -0.010687
proyectos            -0.291606
promedio_horas_mes    0.001839
accidente            -1.246686
ascenso_5years       -1.380084
dep_0                 0.130262
dep_1                 0.211686
dep_2                 0.230800
dep_3                 0.071793
dep_4                -0.220285
dep_5                 0.139730
dep_6                 0.118564
dep_7                 0.020104
dep_8                 0.214053
sal_0                 1.538901
sal_1                 1.147612
Name: coef, dtype: float64

In [31]:
CPH.hazard_ratios_

covariate
satisfaccion          0.784792
evaluacion            0.989370
proyectos             0.747063
promedio_horas_mes    1.001841
accidente             0.287456
ascenso_5years        0.251557
dep_0                 1.139127
dep_1                 1.235759
dep_2                 1.259607
dep_3                 1.074433
dep_4                 0.802290
dep_5                 1.149964
dep_6                 1.125879
dep_7                 1.020307
dep_8                 1.238688
sal_0                 4.659468
sal_1                 3.150662
Name: exp(coef), dtype: float64

De lo anterior tenemos la siguiente prueba de hipótesis con un valor de $\alpha = 0.05$

$H_0 : \beta_{coef} = 0$\
$H_a : \beta_{coef} \neq 0$

$ P_{valor} < \alpha $

---

***satisfacción.***

> Tiene un valor de $ P_{valor} < 0.005 $, entonces no es mayor a $\alpha$

>$ 1 - exp(coef) = 1 - 0.78 = 0.22 $

> Se rechaza $H_0$, por lo que, entre mayor sea la satisfacción del empleado, el riesgo de renuncia disminuye en 22%

---

***evaluación.***

> Tiene un valor de $ P_{valor} = 0.44 $, entonces es mayor a $\alpha$

> No se rechaza $H_0$, por lo que, el desempeño del empleado en su trabajo no afecta en la renuncia del mismo.

---

***proyectos.***

> Tiene un valor de $ P_{valor} < 0.005 $, entonces no es mayor a $\alpha$

>$ 1 - exp(coef) = 1 - 0.75 = 0.25 $

> Se rechaza $H_0$, por lo que, entre mayor proyectos asignados al empleado, el riesgo de renuncia disminuye en 25%

---

***promedio_hora_mes.***

> Tiene un valor de $ P_{valor} < 0.005 $, entonces no es mayor a $\alpha$

>$ 1 - exp(coef) = 1 - 1 = 0 $

>Se rechaza $H_0$, por lo tanto, entre más horas mensuales trabajadas, el riesgo de renuncia aumenta.

---

***accidente.***

> Tiene un valor de $ P_{valor} < 0.005 $, entonces no es mayor a $\alpha$

>$ 1 - exp(coef) = 1 - 0.29 = 0.71 $

 >Se rechaza $H_0$, por lo tanto, si el empleado tuvo accidentes, el riesgo de renuncia disminuye un 71% respecto a si no lo tuvo.

---

**ascenso_5years.****

> Tiene un valor de $ P_{valor} < 0.005 $, entonces no es mayor a $\alpha$

>$ 1 - exp(coef) = 1 - 0.25 = 0.75 $

>Se rechaza $H_0$, por lo tanto, si el empleado tuvo su ascenso en los últimos 5 años, el riesgo de renuncia disminuye en un 75% respecto a si no lo tuvo.

---

***departamentos.***

>**dep_0:** Tiene un valor de $ P_{valor} = 0.37 $, entonces es mayor a $\alpha$

>**dep_1:**Tiene un valor de $ P_{valor} = 0.16 $, entonces es mayor a $\alpha$

>**dep_2:**Tiene un valor de $ P_{valor} = 0.13 $, entonces es mayor a $\alpha$

>**dep_3:**Tiene un valor de $ P_{valor} = 0.66 $, entonces es mayor a $\alpha$

>**dep_4:**Tiene un valor de $ P_{valor} = 0.21 $, entonces es mayor a $\alpha$

>**dep_5:**Tiene un valor de $ P_{valor} = 0.41 $, entonces es mayor a $\alpha$

>**dep_6:**Tiene un valor de $ P_{valor} = 0.48 $, entonces es mayor a $\alpha$

>**dep_7:**Tiene un valor de $ P_{valor} = 0.91 $, entonces es mayor a $\alpha$

>**dep_8:**Tiene un valor de $ P_{valor} = 0.21 $, entonces es mayor a $\alpha$

>Todas las categorias tienen un $P_{valor}>\alpha$, por lo tanto, el departamento de donde es el trabajador no afecta en el riesgo de renuncia.

---

***sal_0.***

> Tiene un valor de $ P_{valor} < 0.005 $, entonces no es mayor a $\alpha$

>$ exp(coef) - 1 = 4.66 - 1 = 3.66 $

>Se rechaza $H_0$, por lo tanto, un salario bajo tieneaumenta el riesgo de renuncia en un 366% respecto al salario alto.

---

***sal_1.***

> Tiene un valor de $ P_{valor} < 0.005 $, entonces no es mayor a $\alpha$

>$ exp(coef) - 1 = 3.15 - 1 = 2.15 $

Se rechaza $H_0$, por lo tanto, un salario medio aumenta el riesgo de renuncia en un 215% respecto al salario alto.

5.Con base  en la interpretación de los coeficientes,¿quérecomendación podría hacerle  a  la  empresa  para  que  disminuya  la  cantidad  de  renuncias  de  los empleados?

Se deberia de checar las encuestas de satisfacción para asi dar un ambiente bueno de trabajo

De igual manera, se podria dar más proyectos a los empleados, hacer que el trabajador trabaje menos horas y dar un ascenso en menos de 5 años a los trabajadores entre otras cosas.

Como se observo que el departamento de investigación es bueno para los empleados se podria revisar que es lo que realizan para poder replicarlo.

Y por último, se debe de subir el salario a los empelados.

6.Pruebe si el modelo de Cox cumple el supuesto de riesgosproporcionales, de no hacerlo estime el modelo AFT adecuado

In [33]:
CPH.check_assumptions(disersion_laboral,p_value_threshold=0.05)

The ``p_value_threshold`` is set at 0.05. Even under the null hypothesis of no violations, some
covariates will be below the threshold by chance. This is compounded when there are many covariates.
Similarly, when there are lots of observations, even minor deviances from the proportional hazard
assumption will be flagged.

With that in mind, it's best to use a combination of statistical tests and visual tests to determine
the most serious violations. Produce visual plots using ``check_assumptions(..., show_plots=True)``
and looking for non-constant lines. See link [A] below for a full example.



<lifelines.StatisticalResult: proportional_hazard_test>
 null_distribution = chi squared
degrees_of_freedom = 1
             model = <lifelines.CoxPHFitter: fitted with 11991 total observations, 10000 right-censored observations>
         test_name = proportional_hazard_test

---
                         test_statistic      p  -log2(p)
accidente          km              0.03   0.87      0.21
                   rank            1.14   0.29      1.81
ascenso_5years     km              3.66   0.06      4.17
                   rank            6.71   0.01      6.71
dep_0              km              3.66   0.06      4.17
                   rank            1.53   0.22      2.21
dep_1              km              4.53   0.03      4.91
                   rank            2.69   0.10      3.31
dep_2              km              6.88   0.01      6.84
                   rank            4.19   0.04      4.62
dep_3              km              3.61   0.06      4.13
                   rank            4.11   0.04      4.55
dep_4              km              5.08   0.02      5.37
                   rank            5.74   0.02      5.92
dep_5              km              2.21   0.14      2.87
                   rank            2.95   0.09      3.54
dep_6              km              0.95   0.33      1.60
                   rank            0.14   0.71      0.49
dep_7              km              4.39   0.04      4.79
                   rank            2.52   0.11      3.16
dep_8              km              2.77   0.10      3.38
                   rank            1.16   0.28      1.82
evaluacion         km            443.05 <0.005    324.32
                   rank          427.15 <0.005    312.82
promedio_horas_mes km            270.84 <0.005    199.74
                   rank          307.41 <0.005    226.21
proyectos          km            574.13 <0.005    419.05
                   rank          684.93 <0.005    499.11
sal_0              km              0.37   0.54      0.88
                   rank            0.00   0.96      0.06
sal_1              km              0.16   0.69      0.53
                   rank            0.10   0.75      0.42
satisfaccion       km           1257.74 <0.005    912.74
                   rank          845.24 <0.005    614.90



1. Variable 'satisfaccion' failed the non-proportional test: p-value is <5e-05.

   Advice 1: the functional form of the variable 'satisfaccion' might be incorrect. That is, there
may be non-linear terms missing. The proportional hazard test used is very sensitive to incorrect
functional forms. See documentation in link [D] below on how to specify a functional form.

   Advice 2: try binning the variable 'satisfaccion' using pd.cut, and then specify it in
`strata=['satisfaccion', ...]` in the call in `.fit`. See documentation in link [B] below.

   Advice 3: try adding an interaction term with your time variable. See documentation in link [C]
below.


2. Variable 'evaluacion' failed the non-proportional test: p-value is <5e-05.

   Advice 1: the functional form of the variable 'evaluacion' might be incorrect. That is, there may
be non-linear terms missing. The proportional hazard test used is very sensitive to incorrect
functional forms. See documentation in link [D] below on how to 

[]

In [35]:
#Modelo alternativo
modelo_exponencial = ExponentialFitter().fit(disersion_laboral['semestre'], disersion_laboral['renunciar'])
modelo_weibull = WeibullFitter().fit(disersion_laboral['semestre'], disersion_laboral['renunciar'])
modelo_logn = LogNormalFitter().fit(disersion_laboral['semestre'], disersion_laboral['renunciar'])
modelo_exponencial.AIC_, modelo_weibull.AIC_, modelo_logn.AIC_

(15965.45892201228, 12511.512867661068, 11587.92169210356)

In [37]:
#Se elige el modelo LogNormal
Logn_AFT = LogNormalAFTFitter().fit(disersion_laboral,'semestre','renunciar')
Logn_AFT.print_summary()

<lifelines.LogNormalAFTFitter: fitted with 11991 total observations, 10000 right-censored observations>
             duration col = 'semestre'
                event col = 'renunciar'
   number of observations = 11991
number of events observed = 1991
           log-likelihood = -4876.57
         time fit was run = 2023-09-26 22:30:54 UTC

---
                            coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
param  covariate                                                                                                                  
mu_    accidente            0.26       1.30       0.02             0.22             0.30                 1.25                 1.35
       ascenso_5years       0.37       1.44       0.06             0.24             0.49                 1.27                 1.63
       dep_0               -0.11       0.90       0.03            -0.17            -0.04                 0.84                 0.96
       dep_1               -0.13       0.88       0.03            -0.19            -0.06                 0.83                 0.94
       dep_2               -0.13       0.88       0.03            -0.20            -0.06                 0.82                 0.94
       dep_3               -0.09       0.91       0.04            -0.17            -0.02                 0.85                 0.98
       dep_4               -0.05       0.95       0.04            -0.13             0.03                 0.88                 1.03
       dep_5               -0.10       0.91       0.04            -0.18            -0.02                 0.84                 0.98
       dep_6               -0.08       0.92       0.04            -0.16            -0.01                 0.85                 0.99
       dep_7               -0.09       0.91       0.04            -0.17            -0.02                 0.85                 0.98
       dep_8               -0.13       0.87       0.04            -0.21            -0.06                 0.81                 0.94
       evaluacion           0.01       1.01       0.00            -0.00             0.01                 1.00                 1.01
       promedio_horas_mes  -0.00       1.00       0.00            -0.00            -0.00                 1.00                 1.00
       proyectos            0.10       1.10       0.01             0.09             0.11                 1.09                 1.11
       sal_0               -0.37       0.69       0.03            -0.43            -0.31                 0.65                 0.73
       sal_1               -0.29       0.75       0.03            -0.34            -0.23                 0.71                 0.80
       satisfaccion         0.07       1.07       0.00             0.06             0.07                 1.06                 1.07
       Intercept            1.39       4.01       0.05             1.30             1.48                 3.67                 4.39
sigma_ Intercept           -1.11       0.33       0.02            -1.14            -1.08                 0.32                 0.34

                            cmp to      z      p   -log2(p)
param  covariate                                           
mu_    accidente              0.00  12.98 <0.005     125.59
       ascenso_5years         0.00   5.80 <0.005      27.19
       dep_0                  0.00  -3.22 <0.005       9.60
       dep_1                  0.00  -3.75 <0.005      12.47
       dep_2                  0.00  -3.85 <0.005      13.04
       dep_3                  0.00  -2.59   0.01       6.70
       dep_4                  0.00  -1.21   0.23       2.14
       dep_5                  0.00  -2.59   0.01       6.71
       dep_6                  0.00  -2.17   0.03       5.04
       dep_7                  0.00  -2.35   0.02       5.75
       dep_8                  0.00  -3.46 <0.005      10.87
       evaluacion             0.00   1.46   0.14       2.80
       promedio_horas_mes     0.00  -2.32   0.02       5.63
    

7.Interprete los coeficientes del modelo AFT

De lo anterior tenemos la siguiente prueba de hipótesis con un valor de $\alpha = 0.05$

$H_0 : \beta_{coef} = 0$\
$H_a : \beta_{coef} \neq 0$

$ P_{valor} < \alpha $

---

***satisfacción.***

> Tiene un valor de $ P_{valor} < 0.005 $, entonces no es mayor a $\alpha$

>$ exp(coef) - 1 = 1.07 - 1 = 0.07 $

> Se rechaza $H_0$, por lo que, entre mayor sea la satisfacción del empleado, el riesgo de renuncia aumenta en 7%

---

***evaluación.***

> Tiene un valor de $ P_{valor} = 0.14 $, entonces es mayor a $\alpha$

> No se rechaza $H_0$, por lo que, el desempeño del empleado en su trabajo no afecta en la renuncia del mismo.

---

***proyectos.***

> Tiene un valor de $ P_{valor} < 0.005 $, entonces no es mayor a $\alpha$

>$ exp(coef) - 1 = 1.10 - 1 = 0.10 $

> Se rechaza $H_0$, por lo que, entre mayor proyectos asignados al empleado, el riesgo de renuncia aumenta en 10%

---

***promedio_hora_mes.***

> Tiene un valor de $ P_{valor} = 0.02 $, entonces no es mayor a $\alpha$

>$ 1 - exp(coef) = 1 - 1 = 0 $

>Se rechaza $H_0$, por lo tanto, entre más horas mensuales trabajadas, el riesgo de renuncia aumenta.

---

***accidente.***

> Tiene un valor de $ P_{valor} < 0.005 $, entonces no es mayor a $\alpha$

>$ exp(coef) - 1 = 1.30 - 1 = 0.30 $

 >Se rechaza $H_0$, por lo tanto, si el empleado sufrió un accidente el riesgo de renuncia aumenta en un 30% respecto a si no lo tuvo

---

**ascenso_5years.****

> Tiene un valor de $ P_{valor} < 0.005 $, entonces no es mayor a $\alpha$

>$ exp(coef) - 1 = 1.44 - 1 = 0.44 $

>Se rechaza $H_0$, por lo tanto, si el empleado fue ascendido, el riesgo de renuncia aumenta en un 44% respecto a si no fue ascendido.

---

***departamentos.***

>**dep_0:** Tiene un valor de $ P_{valor} < 0.005 $, entonces es no mayor a $\alpha$

>**dep_1:**Tiene un valor de $ P_{valor} < 0.005 $, entonces no es mayor a $\alpha$

>**dep_2:**Tiene un valor de $ P_{valor} < 0.005 $, entonces no es mayor a $\alpha$

>**dep_3:**Tiene un valor de $ P_{valor} = 0.01 $, entonces no es mayor a $\alpha$

>**dep_4:**Tiene un valor de $ P_{valor} = 0.23 $, entonces es mayor a $\alpha$

>**dep_5:**Tiene un valor de $ P_{valor} = 0.01 $, entonces no es mayor a $\alpha$

>**dep_6:**Tiene un valor de $ P_{valor} = 0.03 $, entonces no es mayor a $\alpha$

>**dep_7:**Tiene un valor de $ P_{valor} = 0.02 $, entonces no es mayor a $\alpha$

>**dep_8:**Tiene un valor de $ P_{valor} < 0.005$, entonces no es mayor a $\alpha$

>Todas las categorias excepto investigación tienen un $P_{valor}<\alpha$, por lo tanto, se rechaza $H_0$, si el empleado proviene de cualquier departamento que no sea el de investigación, el riesgo de renuncia disminuye entre 5% al 10% respecto a si proviniese del departamento de administración.

>El departamento de investigación no afecta en el riesgo de renuncia.

---

***sal_0.***

> Tiene un valor de $ P_{valor} < 0.005 $, entonces no es mayor a $\alpha$

>$ 1 - exp(coef) = 1 - 0.69 = 0.31 $

>Se rechaza $H_0$, por lo tanto, un salario bajo tiene disminuye el riesgo de renuncia en un 31% respecto al salario alto.

---

***sal_1.***

> Tiene un valor de $ P_{valor} < 0.005 $, entonces no es mayor a $\alpha$

>$ 1 - exp(coef) = 1 - 0.75 = 0.25 $

Se rechaza $H_0$, por lo tanto, un salario medio disminuye el riesgo de renuncia en un 25% respecto al salario alto.